<a href="https://colab.research.google.com/github/avandersonnn/Assignment-1/blob/main/Midterm_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Problem 1 part B

In [2]:
import pandas as pd

opportunities = [
    {"Opportunity": 1, "Investment": 400000, "Return": 100000},
    {"Opportunity": 2, "Investment": 600000, "Return": 180000},
    {"Opportunity": 3, "Investment": 300000, "Return": 60000},
    {"Opportunity": 4, "Investment": 800000, "Return": 400000},
    {"Opportunity": 5, "Investment": 500000, "Return": 200000}
]

budget = 1700000

df = pd.DataFrame(opportunities)

df['Return per Dollar'] = df['Return'] / df['Investment']

df = df.sort_values(by='Return per Dollar', ascending=False).reset_index(drop=True)

total_investment = 0
total_return = 0
allocation = []

for i, row in df.iterrows():
    if total_investment + row['Investment'] <= budget:
        total_investment += row['Investment']
        total_return += row['Return']
        allocation.append({"Opportunity": row['Opportunity'], "Investment": row['Investment'], "Return": row['Return'], "Fraction Invested": 1})
    else:
        remaining_budget = budget - total_investment
        fraction = remaining_budget / row['Investment']
        total_investment += remaining_budget
        total_return += row['Return'] * fraction
        allocation.append({"Opportunity": row['Opportunity'], "Investment": remaining_budget, "Return": row['Return'] * fraction, "Fraction Invested": fraction})
        break

allocation_df = pd.DataFrame(allocation)

print(allocation_df)

from IPython.display import display
display(allocation_df)
total_investment, total_return

   Opportunity  Investment    Return  Fraction Invested
0          4.0    800000.0  400000.0           1.000000
1          5.0    500000.0  200000.0           1.000000
2          2.0    400000.0  120000.0           0.666667


,Opportunity,Investment,Return,Fraction Invested
0,4.0,800000.0,400000.0,1.000000
1,5.0,500000.0,200000.0,1.000000
2,2.0,400000.0,120000.0,0.666667


(1700000.0, 720000.0)

Problem 2 part a

In [3]:
!pip install gurobipy
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 53.7 MB/s eta 0:00:00


In [8]:
import pandas as pd

file_path = '/content/agency matrix.xlsx'

adjacency_matrix = pd.read_excel(file_path, sheet_name='adjacency matrix (2)', header=None)


num_locations = adjacency_matrix.shape[0]
num_warehouses = adjacency_matrix.shape[1]



In [11]:
file_path = '/content/agency matrix.xlsx'

adjacency_matrix = pd.read_excel(file_path, sheet_name='adjacency matrix (2)', header=None)

num_locations = adjacency_matrix.shape[0]
num_warehouses = adjacency_matrix.shape[1]

m = gp.Model("Set_Covering")

x = m.addVars(num_warehouses, vtype=GRB.BINARY, name="Warehouse")

m.setObjective(gp.quicksum(x[j] for j in range(num_warehouses)), GRB.MINIMIZE)

for i in range(num_locations):
    m.addConstr(gp.quicksum(x[j] for j in range(num_warehouses) if adjacency_matrix.iloc[i, j] == 1) >= 1, f"Cover_Location_{i}")


m.optimize()


if m.status == GRB.OPTIMAL:
    print(f"Minimum number of warehouses: {sum(x[j].x for j in range(num_warehouses))}")
    selected_warehouses = [j for j in range(num_warehouses) if x[j].x == 1]
    print(f"Selected warehouses: {selected_warehouses}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 91 rows, 91 columns and 1368 nonzeros
Model fingerprint: 0xc8583dff
Variable types: 0 continuous, 91 integer (91 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 14.0000000
Presolve removed 20 rows and 2 columns
Presolve time: 0.00s
Presolved: 71 rows, 89 columns, 1052 nonzeros
Variable types: 0 continuous, 89 integer (89 binary)

Root relaxation: objective 6.757290e+00, 133 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   

Problem 2 part b

In [12]:
file_path = '/content/agency matrix.xlsx'

adjacency_matrix = pd.read_excel(file_path, sheet_name='adjacency matrix (2)', header=None)

num_locations = adjacency_matrix.shape[0]
num_warehouses = adjacency_matrix.shape[1]

m = gp.Model("Max_Coverage_Limited_Warehouses")

x = m.addVars(num_warehouses, vtype=GRB.BINARY, name="Warehouse")

y = m.addVars(num_locations, vtype=GRB.BINARY, name="LocationCovered")

m.setObjective(gp.quicksum(y[i] for i in range(num_locations)), GRB.MAXIMIZE)

m.addConstr(gp.quicksum(x[j] for j in range(num_warehouses)) <= 4, "Limit_Warehouses")

required_locations = [0, 1, 2, 3]
for loc in required_locations:
    m.addConstr(y[loc] == 1, f"Force_Cover_Location_{loc}")

for i in range(num_locations):
    m.addConstr(gp.quicksum(x[j] for j in range(num_warehouses) if adjacency_matrix.iloc[i, j] == 1) >= y[i], f"Cover_Location_{i}")

m.optimize()

if m.status == GRB.OPTIMAL:
    print(f"Maximum number of locations covered: {sum(y[i].x for i in range(num_locations))}")
    selected_warehouses = [j for j in range(num_warehouses) if x[j].x == 1]
    print(f"Selected warehouses: {selected_warehouses}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 96 rows, 182 columns and 1554 nonzeros
Model fingerprint: 0x088af033
Variable types: 0 continuous, 182 integer (182 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 40.0000000
Presolve removed 24 rows and 23 columns
Presolve time: 0.01s
Presolved: 72 rows, 159 columns, 1211 nonzeros
Variable types: 0 continuous, 159 integer (159 binary)
Found heuristic solution: objective 56.0000000

Root relaxation: objective 7.089831e+01, 206 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Inc

In [13]:
selected_warehouses = [1, 37, 74, 76]
required_locations = [0, 1, 2, 3]

for loc in required_locations:
    covered = False
    for warehouse in selected_warehouses:
        if adjacency_matrix.iloc[loc, warehouse] == 1:
            covered = True
            break
    print(f"Location {loc+1} is covered: {covered}")

Location 1 is covered: True
Location 2 is covered: True
Location 3 is covered: True
Location 4 is covered: True


Problem 3

In [14]:
import gurobipy as gp
from gurobipy import GRB

costs = [
    [7, 17, 12, 11, 8, 12, 7, 14, 13, 13, 15, 16, 11],  # Firm 1
    [7, 9, 8, 14, 15, 10, 9, 17, 8, 11, 11, 12, 10],    # Firm 2
    [12, 11, 17, 18, 10, 15, 11, 13, 9, 8, 16, 8, 15],  # Firm 3
    [9, 14, 9, 13, 14, 10, 6, 13, 9, 16, 11, 8, 14]     # Firm 4
]

num_projects = 13
num_firms = 4

m = gp.Model("Assignment_Problem")

x = m.addVars(num_projects, num_firms, vtype=GRB.BINARY, name="x")

m.setObjective(gp.quicksum(costs[j][i] * x[i, j] for i in range(num_projects) for j in range(num_firms)), GRB.MINIMIZE)

for i in range(num_projects):
    m.addConstr(gp.quicksum(x[i, j] for j in range(num_firms)) == 1, f"Assign_Project_{i}")

for j in range(num_firms):
    m.addConstr(gp.quicksum(x[i, j] for i in range(num_projects)) >= 3, f"Min_Projects_Firm_{j}")

for j in range(num_firms):
    m.addConstr(gp.quicksum(costs[j][i] * x[i, j] for i in range(num_projects)) <= 32, "Max_Cost_Firm_{j}")

m.addConstr(gp.quicksum(x[1, j] * x[2, j] for j in range(num_firms)) == 0, "Conflict_Projects_2_3")

m.optimize()

if m.status == GRB.OPTIMAL:
    print(f"Optimal total cost: {m.objVal * 1000} dollars")
    assignment = {}
    for i in range(num_projects):
        for j in range(num_firms):
            if x[i, j].x > 0.5:
                assignment[f"Project {i+1}"] = f"Firm {j+1}"
    print("Optimal Assignment:")
    for project, firm in assignment.items():
        print(f"{project} is assigned to {firm}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 21 rows, 52 columns and 156 nonzeros
Model fingerprint: 0x3160cd3b
Model has 1 quadratic constraint
Variable types: 0 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  QMatrix range    [1e+00, 1e+00]
  Objective range  [6e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve added 4 rows and 0 columns
Presolve time: 0.00s
Presolved: 25 rows, 52 columns, 164 nonzeros
Variable types: 0 continuous, 52 integer (52 binary)
Found heuristic solution: objective 121.0000000

Root relaxation: objective 1.180000e+02, 20 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntI

Problem 4

In [17]:
mean_constraint = gp.quicksum(i * p[i] for i in support_Z)

model_max.update()
model_max.optimize()

mean_value = mean_constraint.getValue()

variance_expr = gp.QuadExpr()
for i in support_Z:
    variance_expr.add((i - mean_value) * (i - mean_value) * p[i])

model_max.addConstr(variance_expr >= variance_min, "Variance_Min")
model_max.addConstr(variance_expr <= variance_max, "Variance_Max")

model_max.optimize()

if model_max.status == GRB.OPTIMAL:
    max_prob = sum(p[i].x for i in range(14, 18))
    print(f"Maximum probability that Z >= 14 is: {max_prob}")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 18 columns and 35 nonzeros
Model fingerprint: 0x8d58689b
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 8e+00]
Presolve time: 0.01s
Presolved: 2 rows, 18 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7142857e-01   9.000000e-01   0.000000e+00      0s
       2    4.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  4.000000000e-01
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread co

In [19]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

support_Z = list(range(18))

expected_value_Z = 8
variance_min = 25
variance_max = 35
prob_Z_leq_5 = 0.4
max_prob_each_value = 0.1

model_max = gp.Model("Maximize_P_Z_geq_14")

p = model_max.addVars(support_Z, lb=0, ub=max_prob_each_value, name="p")

model_max.setObjective(gp.quicksum(p[i] for i in range(14, 18)), GRB.MAXIMIZE)

model_max.addConstr(gp.quicksum(p[i] for i in support_Z) == 1, "Sum_Prob")

mean_constraint = gp.quicksum(i * p[i] for i in support_Z)
model_max.addConstr(mean_constraint == expected_value_Z, "Expected_Value")

model_max.optimize()

mean_value = mean_constraint.getValue()

variance_expr = gp.QuadExpr()
for i in support_Z:
    variance_expr.add((i - mean_value) * (i - mean_value) * p[i])

model_max.addConstr(variance_expr >= variance_min, "Variance_Min")
model_max.addConstr(variance_expr <= variance_max, "Variance_Max")

model_max.addConstr(gp.quicksum(p[i] for i in range(6)) >= prob_Z_leq_5, "P_Z_leq_5")

model_max.optimize()

if model_max.status == GRB.OPTIMAL:
    max_prob = sum(p[i].x for i in range(14, 18))
    print(f"Maximum probability that Z >= 14 is: {max_prob}")

model_min = gp.Model("Minimize_P_Z_geq_14")

p_min = model_min.addVars(support_Z, lb=0, ub=max_prob_each_value, name="p_min")

model_min.setObjective(gp.quicksum(p_min[i] for i in range(14, 18)), GRB.MINIMIZE)

model_min.addConstr(gp.quicksum(p_min[i] for i in support_Z) == 1, "Sum_Prob_Min")

mean_constraint_min = gp.quicksum(i * p_min[i] for i in support_Z)
model_min.addConstr(mean_constraint_min == expected_value_Z, "Expected_Value_Min")

model_min.optimize()

mean_value_min = mean_constraint_min.getValue()

variance_expr_min = gp.QuadExpr()
for i in support_Z:
    variance_expr_min.add((i - mean_value_min) * (i - mean_value_min) * p_min[i])

model_min.addConstr(variance_expr_min >= variance_min, "Variance_Min_Min")
model_min.addConstr(variance_expr_min <= variance_max, "Variance_Max_Min")

model_min.addConstr(gp.quicksum(p_min[i] for i in range(6)) >= prob_Z_leq_5, "P_Z_leq_5_Min")

model_min.optimize()

if model_min.status == GRB.OPTIMAL:
    min_prob = sum(p_min[i].x for i in range(14, 18))
    print(f"Minimum probability that Z >= 14 is: {min_prob}")


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 2 rows, 18 columns and 35 nonzeros
Model fingerprint: 0x8d58689b
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e-01, 1e-01]
  RHS range        [1e+00, 8e+00]
Presolve time: 0.01s
Presolved: 2 rows, 18 columns, 35 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    5.7142857e-01   9.000000e-01   0.000000e+00      0s
       2    4.0000000e-01   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.02 seconds (0.00 work units)
Optimal objective  4.000000000e-01
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread co

maximum probability that Z>=14 = 37.29%
minimum probability that Z>=14 = 7.46%